### 209

In [4]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
from preprocessing import get_dummies
import datetime

from tqdm import tqdm
import time
import sys
from joblib import Parallel, delayed

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [5]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
# df_hist = utils.read_df_pkl('../input/hist_exist*')
df_hist = reduce_mem_usage(df_hist)
# df_new = utils.read_df_pkl('../input/new_mer*0*')
df_new = utils.read_df_pkl('../input/new_clean_rdm0*')
# df_new = utils.read_df_pkl('../input/new_exist*')
df_new = reduce_mem_usage(df_new)
df_trans = pd.concat([df_hist, df_new], axis=0)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = reduce_mem_usage(df_org_lag)
# df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_trans.shape)
df_trans = df_trans.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_trans.shape)

#========================================================================
# Category
#========================================================================
df_trans = df_trans[df_trans['category_1']==1]
df = df_trans
auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

  0%|          | 0/3 [00:00<?, ?it/s]

Mem. usage decreased to 2554.26 Mb (0.0% reduction)


100%|██████████| 3/3 [00:01<00:00,  2.05it/s]


Mem. usage decreased to 183.47 Mb (0.0% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(31075392, 21)
(31075392, 22)


### New Merchant

In [7]:
auth1['month_lag'].min()

-13

In [9]:
m_id = 'merchant_id'
mc_id = 'merchant_category_id'
lag_col = 'org_month_lag'
lag_col = 'month_lag'
# past_1 = 0
# past_2 = 3
# future_1 = 4
# future_2 = 5

past_2_list = np.array([-1,-2,-3,-4,-5,-6,-7,-8,-9])
past_1_list = np.zeros(len(past_2_list)) -13

future_1_list = past_2_list.copy() + 1
future_2_list = future_1_list.copy() + 1

feat_list = []
fname_list = [ f'auth1_new' ,f'auth0_new']
    

def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

    
def feat_agg(df, fname):
# for df, fname in zip(df_list, fname_list):
    global train_test
    
    aggs = {}
#     if fname.count('cat1'):
#         df = pd.get_dummies(df, columns=['category_3'])
#         aggs = {
#         'category_3_A': ['mean'],
#         'category_3_B': ['mean'],
#         'category_3_C': ['mean'],
#         }
#     else:
#         df = pd.get_dummies(df, columns=['category_2', 'category_3'])
#         aggs = {
#         'category_2_1.0': ['mean'],
#         'category_2_2.0': ['mean'],
#         'category_2_3.0': ['mean'],
#         'category_2_4.0': ['mean'],
#         'category_2_5.0': ['mean'],
#         'category_3_A': ['mean'],
#         'category_3_B': ['mean'],
#         'category_3_C': ['mean'],
#         }
    
#     if not(fname.count('cat')):
#         aggs['category_1'] = ['sum', 'mean']
            
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['first_active_month'] =  pd.to_datetime(df['first_active_month'])
    df['month_diff'] = (pd.to_datetime('2018-05-01') - df['purchase_date']).dt.days
    
#     aggs['month_lag'] = ['mean', 'std']
    aggs['org_month_lag'] = ['mean', 'std']
    aggs['yyyy_week'] = ['nunique']
    aggs['purchase_date'] = ['max','min']
    aggs['month_diff'] = ['mean', 'std']
    
    aggs['purchase_amount'] = ['sum','max', 'min','mean']
    aggs['installments'] = ['sum', 'max', 'min','mean', 'std']
    
    aggs['merchant_id'] = ['nunique']
    aggs['merchant_category_id'] = ['nunique']
    aggs['card_id'] = ['size']
    aggs['city_id'] = ['nunique']
    aggs['state_id'] = ['nunique']
    aggs['subsector_id'] = ['nunique']
    
    new_columns = get_new_columns(fname, aggs)
    df_agg = df.groupby(key).agg(aggs)
    df_agg.columns = new_columns
    df_agg.reset_index(drop=False,inplace=True)
     
    df_agg[f'{fname}_this_term'] =  (pd.to_datetime(df_agg[f'{fname}_purchase_date_max']) - pd.to_datetime(df_agg[f'{fname}_purchase_date_min'])).dt.days
    df_agg[f'{fname}_freq_per_this_term'] = df_agg[f'{fname}_card_id_size']              / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_amount_per_this_term'] = df_agg[f'{fname}_purchase_amount_sum']     / (df_agg[f'{fname}_this_term'] + 1.0)
    df_agg[f'{fname}_instthisments_per_this_term'] = df_agg[f'{fname}_installments_sum'] / (df_agg[f'{fname}_this_term'] + 1.0)
    
    df_agg[f'{fname}_amount_per_installments_sum'] = df_agg[f'{fname}_purchase_amount_sum'] / (df_agg[f'{fname}_installments_sum'] + 1.0)
    df_agg[f'{fname}_amount_per_installments_mean'] = df_agg[f'{fname}_purchase_amount_mean'] / (df_agg[f'{fname}_installments_mean'] + 1.0)
    
    monthly=True
#     monthly=False
    
#     Monthly Agg
    if monthly:
        new_columns = get_new_columns(fname + '_monthly_avg', aggs)
        month_agg = df.groupby([key, 'yyyymm']).agg(aggs)
            
        month_agg.columns = new_columns
        month_agg = month_agg.reset_index().drop('yyyymm', axis=1).groupby([key]).mean()
        month_agg.reset_index(drop=False,inplace=True)
        
        train_test = train_test.join(df_agg.set_index(key)).join(month_agg.set_index(key))
        
        del df_agg, month_agg
        gc.collect()
    else:
        train_test = train_test.join(df_agg.set_index(key))
        del df_agg
        gc.collect()
    
    train_test = impute_agg(train_test)
    train_test.columns =  [col.replace('.', '_') for col in train_test.columns]
    
    display(train_test.head(3))
    
    feat_list.append(train_test.copy())
    del train_test
    gc.collect()
    
#     print(f"Saving features... {fname} {train_test.shape}")
#     p_list = Parallel(n_jobs=-1)([delayed(save_feature)(col) for col in train_test.columns])

#     for col in tqdm(train_test.columns):
#         ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
#         if col in ignore_features: return
#         if (col.count('feature_')):return
#         if (col.count('purchase_date')):return
#         feature = train_test[col].fillna(-1).astype('float32').values
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)


def save_feature(col):
    ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]
    if col in ignore_features: return
    if (col.count('feature_')):return
    if (col.count('purchase_date')):return
    feature = train_test[col].fillna(-1).astype('float32').values
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
    

def impute_agg(df):
    for col in df.columns:
        if col.count('category_1'):
            df[col].fillna(-1, inplace=True)
        if col.count('isnull'):
            df[col].fillna(-1, inplace=True)
        if col.count('null_cnt'):
            df[col].fillna(-1, inplace=True)
        if col.count('month_lag'):
            df[col].fillna(df[col].min()-15, inplace=True)
        if col.count('month_diff'):
            df[col].fillna(df[col].max()+100, inplace=True)
        if col.count('nunique'):
            df[col].fillna(-1, inplace=True)
        if col.count('amount'):
            df[col].fillna(-1, inplace=True)
        if col.count('installments'):
            df[col].fillna(-2, inplace=True)
        if col.count('unix_date'):
            df[col].fillna(df[col].min()-100000, inplace=True)
        if col.count('size'):
            df[col].fillna(-1, inplace=True)
        if col.count('term'):
            df[col].fillna(-1, inplace=True)
        if col.count('per'):
            df[col].fillna(-1, inplace=True)
        if col.count('dummie'):
            df[col].fillna(-1, inplace=True)
            
    return df
    
    
# for df, fname in zip([auth1, auth0], fname_list):
for df, fname_base in zip([auth1], fname_list):
    
    for past_1, past_2 in zip(past_1_list, past_2_list):
        
        future_1 = past_2+1
        future_2 = past_2+2
#         fname = f'180_l{future_1}{future_2}'
        fname = f'_lag{future_1}_{future_2}' + '_' + fname_base

        df_train = utils.read_df_pkl('../input/train0*')
        df_test = utils.read_df_pkl('../input/test0*')
        df_train.set_index(key, inplace=True)
        df_test.set_index(key, inplace=True)
        train_test = pd.concat([df_train, df_test], axis=0)
        train_test.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
        
        future = df[future_1<= df[lag_col]][df[lag_col]<=future_2][[key, mc_id, m_id]].drop_duplicates()
        past = df[past_1<= df[lag_col]][df[lag_col]<=past_2][[key, mc_id, m_id]].drop_duplicates()
        
        # inner joinで過去に存在したmerchantをdf_nowに残し、そのindexをdf_nowから除外することでnew_merchantのみ残す。天才！
        future['index'] = np.arange(len(future))
        mer_same = future.merge(past, how='inner', on=[key, m_id, mc_id])
        past_mer_index = mer_same['index'].values
        future = future.loc[~future['index'].isin(past_mer_index), :]
        
        future.drop('index', axis=1, inplace=True)
        future.dropna(inplace=True)
        
        # ここでtransactionをinner joinしてnew_merchantに対する購買履歴のみのテーブルができる
        new_hist = future.merge(df, how='inner', on=[key, mc_id, m_id])
        
        feat_agg(new_hist, fname)

        
# for df in feat_list:
#     try:
#         df.drop(['feature_1', 'feature_2', 'feature_3', 'first_active_month', 'target'], axis=1, inplace=True)
#     except KeyError:
#         pass
    
#     try:
#         df.set_index(key, inplace=True)
#     except KeyError:
#         pass
    
#     train_test = train_test.join(df)

# train_test = impute_agg(train_test)
# display(train_test.head())

100%|██████████| 3/3 [00:00<00:00, 194.71it/s]


,_lag0_1_auth1_new_org_month_lag_mean,_lag0_1_auth1_new_org_month_lag_std,_lag0_1_auth1_new_yyyy_week_nunique,_lag0_1_auth1_new_purchase_date_max,_lag0_1_auth1_new_purchase_date_min,_lag0_1_auth1_new_month_diff_mean,_lag0_1_auth1_new_month_diff_std,_lag0_1_auth1_new_purchase_amount_sum,_lag0_1_auth1_new_purchase_amount_max,_lag0_1_auth1_new_purchase_amount_min,_lag0_1_auth1_new_purchase_amount_mean,_lag0_1_auth1_new_installments_sum,_lag0_1_auth1_new_installments_max,_lag0_1_auth1_new_installments_min,_lag0_1_auth1_new_installments_mean,_lag0_1_auth1_new_installments_std,_lag0_1_auth1_new_merchant_id_nunique,_lag0_1_auth1_new_merchant_category_id_nunique,_lag0_1_auth1_new_card_id_size,_lag0_1_auth1_new_city_id_nunique,_lag0_1_auth1_new_state_id_nunique,_lag0_1_auth1_new_subsector_id_nunique,_lag0_1_auth1_new_this_term,_lag0_1_auth1_new_freq_per_this_term,_lag0_1_auth1_new_amount_per_this_term,_lag0_1_auth1_new_instthisments_per_this_term,_lag0_1_auth1_new_amount_per_installments_sum,_lag0_1_auth1_new_amount_per_installments_mean,_lag0_1_auth1_new_monthly_avg_org_month_lag_mean,_lag0_1_auth1_new_monthly_avg_org_month_lag_std,_lag0_1_auth1_new_monthly_avg_yyyy_week_nunique,_lag0_1_auth1_new_monthly_avg_month_diff_mean,_lag0_1_auth1_new_monthly_avg_month_diff_std,_lag0_1_auth1_new_monthly_avg_purchase_amount_sum,_lag0_1_auth1_new_monthly_avg_purchase_amount_max,_lag0_1_auth1_new_monthly_avg_purchase_amount_min,_lag0_1_auth1_new_monthly_avg_purchase_amount_mean,_lag0_1_auth1_new_monthly_avg_installments_sum,_lag0_1_auth1_new_monthly_avg_installments_max,_lag0_1_auth1_new_monthly_avg_installments_min,_lag0_1_auth1_new_monthly_avg_installments_mean,_lag0_1_auth1_new_monthly_avg_installments_std,_lag0_1_auth1_new_monthly_avg_merchant_id_nunique,_lag0_1_auth1_new_monthly_avg_merchant_category_id_nunique,_lag0_1_auth1_new_monthly_avg_card_id_size,_lag0_1_auth1_new_monthly_avg_city_id_nunique,_lag0_1_auth1_new_monthly_avg_state_id_nunique,_lag0_1_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-15.0,-15.0,-1.0,NaT,NaT,552.0,143.133514,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-15.0,-15.0,-1.0,552.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_3d0044924f,-15.0,-15.0,-1.0,NaT,NaT,552.0,143.133514,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-15.0,-15.0,-1.0,552.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_d639edf6cd,-15.0,-15.0,-1.0,NaT,NaT,552.0,143.133514,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-15.0,-15.0,-1.0,552.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


100%|██████████| 3/3 [00:00<00:00, 131.71it/s]


,_lag-1_0_auth1_new_org_month_lag_mean,_lag-1_0_auth1_new_org_month_lag_std,_lag-1_0_auth1_new_yyyy_week_nunique,_lag-1_0_auth1_new_purchase_date_max,_lag-1_0_auth1_new_purchase_date_min,_lag-1_0_auth1_new_month_diff_mean,_lag-1_0_auth1_new_month_diff_std,_lag-1_0_auth1_new_purchase_amount_sum,_lag-1_0_auth1_new_purchase_amount_max,_lag-1_0_auth1_new_purchase_amount_min,_lag-1_0_auth1_new_purchase_amount_mean,_lag-1_0_auth1_new_installments_sum,_lag-1_0_auth1_new_installments_max,_lag-1_0_auth1_new_installments_min,_lag-1_0_auth1_new_installments_mean,_lag-1_0_auth1_new_installments_std,_lag-1_0_auth1_new_merchant_id_nunique,_lag-1_0_auth1_new_merchant_category_id_nunique,_lag-1_0_auth1_new_card_id_size,_lag-1_0_auth1_new_city_id_nunique,_lag-1_0_auth1_new_state_id_nunique,_lag-1_0_auth1_new_subsector_id_nunique,_lag-1_0_auth1_new_this_term,_lag-1_0_auth1_new_freq_per_this_term,_lag-1_0_auth1_new_amount_per_this_term,_lag-1_0_auth1_new_instthisments_per_this_term,_lag-1_0_auth1_new_amount_per_installments_sum,_lag-1_0_auth1_new_amount_per_installments_mean,_lag-1_0_auth1_new_monthly_avg_org_month_lag_mean,_lag-1_0_auth1_new_monthly_avg_org_month_lag_std,_lag-1_0_auth1_new_monthly_avg_yyyy_week_nunique,_lag-1_0_auth1_new_monthly_avg_month_diff_mean,_lag-1_0_auth1_new_monthly_avg_month_diff_std,_lag-1_0_auth1_new_monthly_avg_purchase_amount_sum,_lag-1_0_auth1_new_monthly_avg_purchase_amount_max,_lag-1_0_auth1_new_monthly_avg_purchase_amount_min,_lag-1_0_auth1_new_monthly_avg_purchase_amount_mean,_lag-1_0_auth1_new_monthly_avg_installments_sum,_lag-1_0_auth1_new_monthly_avg_installments_max,_lag-1_0_auth1_new_monthly_avg_installments_min,_lag-1_0_auth1_new_monthly_avg_installments_mean,_lag-1_0_auth1_new_monthly_avg_installments_std,_lag-1_0_auth1_new_monthly_avg_merchant_id_nunique,_lag-1_0_auth1_new_monthly_avg_merchant_category_id_nunique,_lag-1_0_auth1_new_monthly_avg_card_id_size,_lag-1_0_auth1_new_monthly_avg_city_id_nunique,_lag-1_0_auth1_new_monthly_avg_state_id_nunique,_lag-1_0_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-16.666667,-15.0,-1.0,NaT,NaT,576.0,142.426407,-1.000000,-1.000000,-1.000000,-1.000000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-16.5,-15.0,-1.0,576.0,121.213203,-1.000000,-1.000000,-1.000000,-1.000000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_3d0044924f,11.000000,0.0,3.0,2017-12-19 18:23:10,2017-12-05 11:55:27,138.0,7.211103,1.095703,1.083984,0.004517,0.365234,5.0,3.0,1.0,1.666992,1.154297,3.0,3.0,3.0,1.0,1.0,2.0,14.0,0.2,0.073047,0.333333,0.182617,0.136841,11.0,0.0,3.0,138.0,7.211103,1.095703,1.083984,0.004517,0.365234,5.0,3.0,1.0,1.666992,1.154297,3.0,3.0,3.0,1.0,1.0,2.0
C_ID_d639edf6cd,-16.666667,-15.0,-1.0,NaT,NaT,576.0,142.426407,-1.000000,-1.000000,-1.000000,-1.000000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-16.5,-15.0,-1.0,576.0,121.213203,-1.000000,-1.000000,-1.000000,-1.000000,-2.0,-2.0,-2.0,-2.000000,-2.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


100%|██████████| 3/3 [00:00<00:00, 134.68it/s]


...

,_lag-6_-5_auth1_new_org_month_lag_mean,_lag-6_-5_auth1_new_org_month_lag_std,_lag-6_-5_auth1_new_yyyy_week_nunique,_lag-6_-5_auth1_new_purchase_date_max,_lag-6_-5_auth1_new_purchase_date_min,_lag-6_-5_auth1_new_month_diff_mean,_lag-6_-5_auth1_new_month_diff_std,_lag-6_-5_auth1_new_purchase_amount_sum,_lag-6_-5_auth1_new_purchase_amount_max,_lag-6_-5_auth1_new_purchase_amount_min,_lag-6_-5_auth1_new_purchase_amount_mean,_lag-6_-5_auth1_new_installments_sum,_lag-6_-5_auth1_new_installments_max,_lag-6_-5_auth1_new_installments_min,_lag-6_-5_auth1_new_installments_mean,_lag-6_-5_auth1_new_installments_std,_lag-6_-5_auth1_new_merchant_id_nunique,_lag-6_-5_auth1_new_merchant_category_id_nunique,_lag-6_-5_auth1_new_card_id_size,_lag-6_-5_auth1_new_city_id_nunique,_lag-6_-5_auth1_new_state_id_nunique,_lag-6_-5_auth1_new_subsector_id_nunique,_lag-6_-5_auth1_new_this_term,_lag-6_-5_auth1_new_freq_per_this_term,_lag-6_-5_auth1_new_amount_per_this_term,_lag-6_-5_auth1_new_instthisments_per_this_term,_lag-6_-5_auth1_new_amount_per_installments_sum,_lag-6_-5_auth1_new_amount_per_installments_mean,_lag-6_-5_auth1_new_monthly_avg_org_month_lag_mean,_lag-6_-5_auth1_new_monthly_avg_org_month_lag_std,_lag-6_-5_auth1_new_monthly_avg_yyyy_week_nunique,_lag-6_-5_auth1_new_monthly_avg_month_diff_mean,_lag-6_-5_auth1_new_monthly_avg_month_diff_std,_lag-6_-5_auth1_new_monthly_avg_purchase_amount_sum,_lag-6_-5_auth1_new_monthly_avg_purchase_amount_max,_lag-6_-5_auth1_new_monthly_avg_purchase_amount_min,_lag-6_-5_auth1_new_monthly_avg_purchase_amount_mean,_lag-6_-5_auth1_new_monthly_avg_installments_sum,_lag-6_-5_auth1_new_monthly_avg_installments_max,_lag-6_-5_auth1_new_monthly_avg_installments_min,_lag-6_-5_auth1_new_monthly_avg_installments_mean,_lag-6_-5_auth1_new_monthly_avg_installments_std,_lag-6_-5_auth1_new_monthly_avg_merchant_id_nunique,_lag-6_-5_auth1_new_monthly_avg_merchant_category_id_nunique,_lag-6_-5_auth1_new_monthly_avg_card_id_size,_lag-6_-5_auth1_new_monthly_avg_city_id_nunique,_lag-6_-5_auth1_new_monthly_avg_state_id_nunique,_lag-6_-5_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-19.0,-15.0,-1.0,NaT,NaT,584.0,247.785317,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-19.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_3d0044924f,-19.0,-15.0,-1.0,NaT,NaT,584.0,247.785317,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-19.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_d639edf6cd,-19.0,-15.0,-1.0,NaT,NaT,584.0,247.785317,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-19.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


100%|██████████| 3/3 [00:00<00:00, 188.32it/s]


,_lag-7_-6_auth1_new_org_month_lag_mean,_lag-7_-6_auth1_new_org_month_lag_std,_lag-7_-6_auth1_new_yyyy_week_nunique,_lag-7_-6_auth1_new_purchase_date_max,_lag-7_-6_auth1_new_purchase_date_min,_lag-7_-6_auth1_new_month_diff_mean,_lag-7_-6_auth1_new_month_diff_std,_lag-7_-6_auth1_new_purchase_amount_sum,_lag-7_-6_auth1_new_purchase_amount_max,_lag-7_-6_auth1_new_purchase_amount_min,_lag-7_-6_auth1_new_purchase_amount_mean,_lag-7_-6_auth1_new_installments_sum,_lag-7_-6_auth1_new_installments_max,_lag-7_-6_auth1_new_installments_min,_lag-7_-6_auth1_new_installments_mean,_lag-7_-6_auth1_new_installments_std,_lag-7_-6_auth1_new_merchant_id_nunique,_lag-7_-6_auth1_new_merchant_category_id_nunique,_lag-7_-6_auth1_new_card_id_size,_lag-7_-6_auth1_new_city_id_nunique,_lag-7_-6_auth1_new_state_id_nunique,_lag-7_-6_auth1_new_subsector_id_nunique,_lag-7_-6_auth1_new_this_term,_lag-7_-6_auth1_new_freq_per_this_term,_lag-7_-6_auth1_new_amount_per_this_term,_lag-7_-6_auth1_new_instthisments_per_this_term,_lag-7_-6_auth1_new_amount_per_installments_sum,_lag-7_-6_auth1_new_amount_per_installments_mean,_lag-7_-6_auth1_new_monthly_avg_org_month_lag_mean,_lag-7_-6_auth1_new_monthly_avg_org_month_lag_std,_lag-7_-6_auth1_new_monthly_avg_yyyy_week_nunique,_lag-7_-6_auth1_new_monthly_avg_month_diff_mean,_lag-7_-6_auth1_new_monthly_avg_month_diff_std,_lag-7_-6_auth1_new_monthly_avg_purchase_amount_sum,_lag-7_-6_auth1_new_monthly_avg_purchase_amount_max,_lag-7_-6_auth1_new_monthly_avg_purchase_amount_min,_lag-7_-6_auth1_new_monthly_avg_purchase_amount_mean,_lag-7_-6_auth1_new_monthly_avg_installments_sum,_lag-7_-6_auth1_new_monthly_avg_installments_max,_lag-7_-6_auth1_new_monthly_avg_installments_min,_lag-7_-6_auth1_new_monthly_avg_installments_mean,_lag-7_-6_auth1_new_monthly_avg_installments_std,_lag-7_-6_auth1_new_monthly_avg_merchant_id_nunique,_lag-7_-6_auth1_new_monthly_avg_merchant_category_id_nunique,_lag-7_-6_auth1_new_monthly_avg_card_id_size,_lag-7_-6_auth1_new_monthly_avg_city_id_nunique,_lag-7_-6_auth1_new_monthly_avg_state_id_nunique,_lag-7_-6_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-21.0,-15.0,-1.0,NaT,NaT,584.0,270.412734,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-21.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_3d0044924f,-21.0,-15.0,-1.0,NaT,NaT,584.0,270.412734,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-21.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_d639edf6cd,-21.0,-15.0,-1.0,NaT,NaT,584.0,270.412734,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-21.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


100%|██████████| 3/3 [00:00<00:00, 139.88it/s]


,_lag-8_-7_auth1_new_org_month_lag_mean,_lag-8_-7_auth1_new_org_month_lag_std,_lag-8_-7_auth1_new_yyyy_week_nunique,_lag-8_-7_auth1_new_purchase_date_max,_lag-8_-7_auth1_new_purchase_date_min,_lag-8_-7_auth1_new_month_diff_mean,_lag-8_-7_auth1_new_month_diff_std,_lag-8_-7_auth1_new_purchase_amount_sum,_lag-8_-7_auth1_new_purchase_amount_max,_lag-8_-7_auth1_new_purchase_amount_min,_lag-8_-7_auth1_new_purchase_amount_mean,_lag-8_-7_auth1_new_installments_sum,_lag-8_-7_auth1_new_installments_max,_lag-8_-7_auth1_new_installments_min,_lag-8_-7_auth1_new_installments_mean,_lag-8_-7_auth1_new_installments_std,_lag-8_-7_auth1_new_merchant_id_nunique,_lag-8_-7_auth1_new_merchant_category_id_nunique,_lag-8_-7_auth1_new_card_id_size,_lag-8_-7_auth1_new_city_id_nunique,_lag-8_-7_auth1_new_state_id_nunique,_lag-8_-7_auth1_new_subsector_id_nunique,_lag-8_-7_auth1_new_this_term,_lag-8_-7_auth1_new_freq_per_this_term,_lag-8_-7_auth1_new_amount_per_this_term,_lag-8_-7_auth1_new_instthisments_per_this_term,_lag-8_-7_auth1_new_amount_per_installments_sum,_lag-8_-7_auth1_new_amount_per_installments_mean,_lag-8_-7_auth1_new_monthly_avg_org_month_lag_mean,_lag-8_-7_auth1_new_monthly_avg_org_month_lag_std,_lag-8_-7_auth1_new_monthly_avg_yyyy_week_nunique,_lag-8_-7_auth1_new_monthly_avg_month_diff_mean,_lag-8_-7_auth1_new_monthly_avg_month_diff_std,_lag-8_-7_auth1_new_monthly_avg_purchase_amount_sum,_lag-8_-7_auth1_new_monthly_avg_purchase_amount_max,_lag-8_-7_auth1_new_monthly_avg_purchase_amount_min,_lag-8_-7_auth1_new_monthly_avg_purchase_amount_mean,_lag-8_-7_auth1_new_monthly_avg_installments_sum,_lag-8_-7_auth1_new_monthly_avg_installments_max,_lag-8_-7_auth1_new_monthly_avg_installments_min,_lag-8_-7_auth1_new_monthly_avg_installments_mean,_lag-8_-7_auth1_new_monthly_avg_installments_std,_lag-8_-7_auth1_new_monthly_avg_merchant_id_nunique,_lag-8_-7_auth1_new_monthly_avg_merchant_category_id_nunique,_lag-8_-7_auth1_new_monthly_avg_card_id_size,_lag-8_-7_auth1_new_monthly_avg_city_id_nunique,_lag-8_-7_auth1_new_monthly_avg_state_id_nunique,_lag-8_-7_auth1_new_monthly_avg_subsector_id_nunique
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,-22.0,-15.0,-1.0,NaT,NaT,584.0,291.625938,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-22.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_3d0044924f,-22.0,-15.0,-1.0,NaT,NaT,584.0,291.625938,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-22.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
C_ID_d639edf6cd,-22.0,-15.0,-1.0,NaT,NaT,584.0,291.625938,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-22.0,-15.0,-1.0,584.0,121.213203,-1.0,-1.0,-1.0,-1.0,-2.0,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [10]:
#========================================================================
# Save Feature
#========================================================================

ignore_features = ['unix_first_active_month', 'first_active_month', 'card_id', target]

for df_feat, fname in zip(feat_list[1:], ['l0_1', 'l-1_0', 'lag-2_-1', 'l-3_-2', 'l-4_-3', 'l-5_-3', 'l-6_-4']):
#     fname = '209_' + fname
    fname = '210'
    for col in df_feat.columns:
        if col in ignore_features: continue
        if (col.count('feature_')):continue
        if (col.count('purchase_date')) and not(col.count('diff')):continue
        feature = df_feat[col].fillna(-1).astype('float32').values
#         utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
        utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
#         utils.to_pkl_gzip(path = f'../features/5_tmp/{fname}_{col}@', obj=feature)

In [12]:
import glob
import shutil
path_list = glob.glob('../features/4_winner/*.gz')

for path in path_list:
    tmp = utils.read_pkl_gzip(path)
    if tmp.shape[0]>330000:
        shutil.move(path, '../features/6_subset/')